In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from pandas.tseries.offsets import BDay
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
from statsmodels.tsa.stattools import adfuller, kpss
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

pd.set_option("display.max_rows", None, "display.max_columns", None)

#LoadCSV
df = pd.read_csv(r'C:\Users\asousa\Desktop\Produtech\PPS4\AnáliseDados\dadospps4\Encomendas\combined_csv.csv', sep=";")
#Dias2020
df2020 = pd.read_csv(r'C:\Users\asousa\Desktop\Produtech\PPS4\AnáliseDados\dadospps4\Encomendas\D2020.csv')

#Formatação Dataframe
df = df.drop(columns=['Sold-To Party', 'Name 1', 'Sales Document', 'Purchase order no.', 'Material', 'Description', 'Delivery Date', 'Created By'])

df['Order Quantity'] = df['Order Quantity'].str.replace(",", ".")
df['Order Quantity'] = pd.to_numeric(df['Order Quantity'])

df2020['Dias'] = pd.to_datetime(df2020['Dias'])
df2020 = df2020.sort_values(by='Dias').reset_index(drop=True)

df['Document Date'] = pd.to_datetime(df['Document Date'])

#Groupby Dia
df_gb = df.groupby(df['Document Date'].dt.date).sum().reset_index()

df = pd.DataFrame(df_gb)
df2020 = pd.DataFrame(df2020)

#Dias uteis [df]
isBusinessDay = BDay().is_on_offset
dias_semana = pd.to_datetime(df['Document Date']).map(isBusinessDay)
df["dias_semana"] = dias_semana

#Divisao em dataframes com os dias uteis e fins de semana [df]
df_smn = df[df['dias_semana'] == 1].reset_index(drop=True)
df_fs = df[df['dias_semana'] == 0]

#Dias uteis [df2020]
work_day = pd.to_datetime(df2020['Dias']).map(isBusinessDay)
df2020['work_day'] = work_day

#Divisao em dataframes com os dias uteis e fins de semana [df2020]
df_smn_20 = df2020[df2020['work_day'] == 1].reset_index(drop=True)
df_fs_20 = df2020[df2020['work_day'] == 0]


#Formataçao
df_smn_20.rename(columns={"Dias" : "Document Date"}, inplace=True)
df_smn_20["Document Date"] = df_smn_20["Document Date"].astype("str")

df_test = pd.merge(df_smn, df_smn_20, on="Document Date", how="outer")
df_test["Document Date"]  = pd.to_datetime(df_test["Document Date"])
df_test.sort_values(by='Document Date').reset_index(drop=True)

df_test = df_test.groupby(['Document Date']).apply(lambda x: x[~x['Document Date'].duplicated() | ~x['Order Quantity'].isna()]) 
df = df_test.reset_index(drop=True)
df = df.drop(columns=["dias_semana", "work_day"])
df['Order Quantity'] = df['Order Quantity'].fillna(0)

df = df.drop(columns=['Document Date'])

In [36]:
#AUTO ARIMA

from pmdarima.arima import auto_arima
from pmdarima.arima import ADFTest
from sklearn.model_selection import train_test_split

#df['Document Date']=pd.to_datetime(df['Document Date'])
#df.set_index('Document Date',inplace=True)

#adf_test = ADFTest(alpha = 0.05)
#adf_test.should_diff(df)

#train = df[3:228]
#test = df[228:258]

#train, test = train_test_split(df, test_size=0.2)

In [37]:
#arima_model =  auto_arima(train, start_p=0, d=1, start_q=0, 
                          max_p=5, max_d=5, max_q=5, start_P=0, 
                          D=1, start_Q=0, max_P=5, max_D=5,
                          max_Q=5, m=12, seasonal=True, 
                          error_action='warn',trace = True,
                          supress_warnings=True,stepwise = True,
                          random_state=20,n_fits = 50)

IndentationError: unexpected indent (Temp/ipykernel_17204/17803169.py, line 2)

In [13]:
#arima_model

NameError: name 'arima_model' is not defined

In [38]:
def hold(y, ratio=2 / 3, mode="rolling", it=1, window=10, increment=1):
    #ratio = test size
    #window = traingin size
    ALLITR = None
    VAL = None
    NSIZE = len(y)
    if mode == "incremental" or mode == "rolling":
        aux = window + increment * (it - 1)
        aux = min(aux, NSIZE)
        if mode == "rolling":
            iaux = max((aux - window + 1), 1)
        else:
            iaux = 1
        ALLTR = range(iaux, aux)
        end = aux + ratio
        end = min(end, NSIZE)
        iend = aux + 1
        if iend < end:
            TS = range(iend, end)
        else:
            TS = None
    return {"tr": ALLTR, "itr": ALLITR, "val": VAL, "ts": TS}

In [39]:
def calcIterationsRW(datasetSize, trainingSize, testSize, stepSize): 
    
    return int((datasetSize - (trainingSize + testSize))/ stepSize)

trainingSize = 130
testSize = 6
ratio = testSize
stepSize = 6
#totalLenght = df.shape[0]
mode = "rolling"

iterations = calcIterationsRW(df.shape[0], trainingSize, testSize, stepSize)

#For only positive records
#iterations = calcIterationsRW(len(df['serial'].unique()), trainingSize, testSize, stepSize)

iterations

21

In [40]:
from pmdarima.arima import auto_arima
from tqdm import trange
from sklearn.metrics import mean_absolute_error

def runarima(data, iterations):
    max_value = data['Order Quantity'].max()
    min_value = data['Order Quantity'].min()
    denominator = max_value - min_value
    
    dfFinal = pd.DataFrame(columns = ['MAE', 'NMAE'])
    
    MAEVals = []
    NMAEVals = []
   
    for i in trange(iterations):

            holds = hold(data[['Order Quantity']], ratio, mode, i+1, trainingSize, stepSize)
            print('holds')
            print("Iteration ", i+1 , "of ", iterations, " TR: ", holds['tr'] , "TS: ", holds['ts'])
            tr = data.iloc[holds['tr']].reset_index(drop=True)
            ts = data.iloc[holds['ts']].reset_index(drop=True)
            
            model =  auto_arima(tr[["Order Quantity"]], start_p=0, d=1, start_q=0, 
                          max_p=5, max_d=5, max_q=5, start_P=0, 
                          D=1, start_Q=0, max_P=5, max_D=5,
                          max_Q=5, m=12, seasonal=True, 
                          error_action='warn',trace = True,
                          supress_warnings=True,stepwise = True,
                          random_state=20,n_fits = 20)
            
            p = model.predict(n_periods=5)
            MAE = mean_absolute_error(ts.values.tolist(), p)
            NMAE = (MAE/denominator)*100 #o denominador é calculado com o maximo e minimo global
            
            print("Iteration ", i , "MAE ", MAE, "NMAE of ", NMAE)
            
            MAEVals.append(MAE)
            NMAEVals.append(NMAE)
        
            new = {'MAE': MAE, 'NMAE': NMAE}
            dfFinal = dfFinal.append(new, ignore_index=True)
            dfFinal = pd.DataFrame(dfFinal)
            
    return dfFinal

In [41]:
dfResults = runarima(df,iterations)

  0%|          | 0/21 [00:00<?, ?it/s]

holds
Iteration  1 of  21  TR:  range(1, 130) TS:  range(131, 136)
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=2761.962, Time=0.02 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=2695.497, Time=0.07 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=0.23 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=2733.016, Time=0.02 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=2690.346, Time=0.11 sec
 ARIMA(1,1,0)(3,1,0)[12]             : AIC=2690.678, Time=0.23 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=2688.487, Time=0.22 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=2686.516, Time=0.10 sec
 ARIMA(1,1,0)(0,1,1)[12]             : AIC=2687.051, Time=0.07 sec
 ARIMA(1,1,0)(1,1,2)[12]             : AIC=2688.493, Time=0.39 sec
 ARIMA(1,1,0)(0,1,2)[12]             : AIC=2686.790, Time=0.15 sec
 ARIMA(1,1,0)(2,1,2)[12]             : AIC=2690.447, Time=0.54 sec
 ARIMA(0,1,0)(1,1,1)[12]             : AIC=2697.965, Time=0.07 sec
 ARIMA(2,1,0)(1,1,1)[12]

  5%|▍         | 1/21 [00:32<10:48, 32.45s/it]

 ARIMA(4,1,2)(1,1,1)[12] intercept   : AIC=2658.986, Time=0.65 sec

Best model:  ARIMA(4,1,2)(1,1,1)[12]          
Total fit time: 32.357 seconds
Iteration  0 MAE  8978.318190011298 NMAE of  9.823668435708239
holds
Iteration  2 of  21  TR:  range(7, 136) TS:  range(137, 142)
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=2768.773, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=2696.273, Time=0.05 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=0.21 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=2736.128, Time=0.02 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=2689.036, Time=0.12 sec
 ARIMA(1,1,0)(3,1,0)[12]             : AIC=2689.063, Time=0.23 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=2687.011, Time=0.26 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=2685.011, Time=0.10 sec
 ARIMA(1,1,0)(0,1,1)[12]             : AIC=2685.448, Time=0.05 sec
 ARIMA(1,1,0)(1,1,2)[12]             : AIC=2687.011, Time=0.31 sec
 ARIMA(1,1,0)(0,

 10%|▉         | 2/21 [00:50<07:35, 24.00s/it]

...

 ARIMA(2,1,3)(2,1,2)[12]             : AIC=2567.339, Time=2.00 sec

Best model:  ARIMA(3,1,3)(2,1,2)[12]          
Total fit time: 159.194 seconds
Iteration  18 MAE  18253.722324799117 NMAE of  19.97239483401359
holds
Iteration  20 of  21  TR:  range(115, 244) TS:  range(245, 250)
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=2642.892, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=2596.666, Time=0.07 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=0.23 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=2626.075, Time=0.02 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=2582.367, Time=0.18 sec
 ARIMA(1,1,0)(3,1,0)[12]             : AIC=2583.445, Time=0.29 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=2581.742, Time=0.34 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=2581.378, Time=0.11 sec
 ARIMA(1,1,0)(0,1,1)[12]             : AIC=2579.598, Time=0.09 sec
 ARIMA(1,1,0)(0,1,2)[12]             : AIC=2581.274, Time=0.19 sec
 ARIMA(1,1

 95%|█████████▌| 20/21 [11:49<00:50, 50.88s/it]

 ARIMA(1,1,3)(0,1,1)[12] intercept   : AIC=2546.960, Time=0.51 sec

Best model:  ARIMA(1,1,3)(0,1,1)[12]          
Total fit time: 16.583 seconds
Iteration  19 MAE  14240.711260687953 NMAE of  15.581540189708857
holds
Iteration  21 of  21  TR:  range(121, 250) TS:  range(251, 256)
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=2657.442, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=2609.798, Time=0.08 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=2570.097, Time=0.11 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=2614.640, Time=0.04 sec
 ARIMA(0,1,1)(1,1,1)[12]             : AIC=2570.598, Time=0.16 sec
 ARIMA(0,1,1)(0,1,2)[12]             : AIC=2570.098, Time=0.28 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=2583.010, Time=0.14 sec
 ARIMA(0,1,1)(1,1,2)[12]             : AIC=2571.781, Time=0.44 sec
 ARIMA(0,1,0)(0,1,1)[12]             : AIC=2605.431, Time=0.06 sec
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=2571.184, Time=0.18 sec
 ARIM

100%|██████████| 21/21 [12:06<00:00, 34.62s/it]

 ARIMA(0,1,4)(0,1,1)[12] intercept   : AIC=2574.291, Time=0.30 sec

Best model:  ARIMA(0,1,4)(0,1,1)[12]          
Total fit time: 17.621 seconds
Iteration  20 MAE  13607.580804549809 NMAE of  14.888797568427128


from pmdarima.arima import auto_arima
def runarima(data, iterations, model):
    max_value = data['Delta'].max()
    min_value = data['Delta'].min()
    denominator = max_value - min_value
    for i in trange(iterations):

            holds = hold(data[['Delta']], ratio, mode, i+1, trainingSize, stepSize)
            print('holds')
            print("Iteration ", i , "of ", iterations, " TR: ", holds['tr'] , "TS: ", holds['ts'])
            tr = data.iloc[holds['tr']].reset_index(drop=True)
            ts = data.iloc[holds['ts']].reset_index(drop=True)
            
            model.fit(tr)
            
            p = model.predict(ts[["Delta"]])
            
            NMAE = (MAE/denominator)*100 #o denominador é calculado com o maximo e minimo global
            
            print("Iteration ", i , "NMAE of ", NMAE)


In [46]:
#prediction = pd.DataFrame(model.predict(n_periods = len(test)), index=test.index)
dfResults.columns = ['predicted_sales']

ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements

In [ ]:
#from sklearn.metrics import mean_absolute_error

#mae = mean_absolute_error(test, prediction)
#nmae = 100 * (mae / abs(test.max() - test.min()))

In [44]:
mae_mean = dfResults['MAE'].mean()
nmae_mean = dfResults['NMAE'].mean()
print(mae_mean)
print(nmae_mean)

11036.19896643179
12.075308219455675


In [ ]:
nmae

In [42]:
dfResults

,MAE,NMAE
0,8978.318190,9.823668
1,9087.530222,9.943163
2,12053.103564,13.187959
3,11982.726110,13.110955
4,12209.091192,13.358634
5,12517.799090,13.696408
6,7566.937952,8.279400
7,4761.694950,5.210031
8,21381.844576,23.395044
9,9324.014960,10.201914
